# Offshore wind farms

This notebook pulls wind farm data from OpenStreetMap (OSM) and plots them in Bokeh plots. The idea is to produce something interactive and fun to play with. 

## Setup

Import libraries and standard tools used

In [1]:
import os
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource, OpenURL, TapTool
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.palettes import viridis, Category20

from pyproj import Transformer

output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
area_UK = "(49.325, -11.25, 59.356, 1.95)"
area = "(51.4,2.6,51.9,3.2)" # Borselle etc.
#area = "(51.55,1.4,51.75,1.6)" # London Array


In [4]:
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

In [5]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")
 

In [6]:
# Place to save data so that the number of OSM requests are reduced when re-loading the same data
path_save = r"data/"

if not os.path.exists(path_save):
    os.makedirs(path_save)
    

In [7]:
def get_power_lines(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "power"
    tag = "cable"
    output = "geom"
    recursion = ""
    element="way"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    return data

In [8]:
def get_wind_turbines(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "generator:method"
    tag = "wind_turbine"
    output = "center"
    recursion = ""
    element = "node"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    return data

In [9]:
def get_substations(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "power"
    tag = "substation"
    output = "center"
    recursion = ""
    element = "node"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    return data

In [10]:
def get_plants(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "plant:source"
    tag = "wind"
    output = "geom"
    recursion = ""
    element = "way"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    return data

In [11]:
def get_power_line_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df = pd.read_pickle(file_name)
    else:
        df = get_power_lines(area)
        df.to_pickle(file_name)
        
    return df


In [12]:
def get_wind_turbine_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df = pd.read_pickle(file_name)
    else:
        df = get_wind_turbines(area)
        df.to_pickle(file_name)
        
    return df

In [13]:
def get_substation_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df = pd.read_pickle(file_name)
    else:
        df = get_substations(area)
        df.to_pickle(file_name)
        
    return df

In [14]:
def get_plant_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df = pd.read_pickle(file_name)
    else:
        df = get_plants(area)
        df.to_pickle(file_name)
        
    return df

In [15]:
def plot_wind_farms(df_wind_turbines,df_power_lines,df_substations,kwargs_for_figure={}):

    # Define default and then update figure and marker options based on kwargs
    figure_options = {
        "tools": "save,pan,wheel_zoom,reset,help",
        "x_axis_label": "Longitude",
        "y_axis_label": "Latitude",
        "match_aspect": True,
    }
    figure_options.update(kwargs_for_figure)

    # Create a bokeh figure with tiles
    p = figure(
        width=800,
        height=800,
        x_axis_type="mercator",
        y_axis_type="mercator",
        **figure_options,
    )

    p.add_tile(MAP_TILES['ESRI'])

    if df_power_lines is not None:
        xx = list()
        yy = list()
        names = list()
        voltage = list()
        colors = list()
        for cnt,way in df_power_lines.iterrows():


            xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way['geometry'])['lat']),
                                                    list(pd.DataFrame.from_records(way['geometry'])['lon']))

            xx.append(xxx)
            yy.append(yyy)
            voltage.append(way['voltage'])
            colors.append(way['color'])


        source = ColumnDataSource({'xs':xx,'ys':yy,'color':colors,'voltage':voltage})

        render_lines = p.multi_line('xs',
                    'ys',
                    source=source,
                    line_width=2,
                    line_color='color',
            )

        # HoverTool for power lines
        hover_lines = HoverTool(renderers=[render_lines])
        hover_lines.tooltips = [("Voltage", "@voltage")]
        p.add_tools(hover_lines)



    # Plot the wind turbines
    marker_options = {
        "marker": "circle_y",
        "line_width": 2,
        "alpha": 0.5,
        "fill_color": "blue",
        "line_color": "white",
        #"legend_group": "data_rank",
    }

    df_wind_turbines["x"], df_wind_turbines["y"] = TRANSFORM_4326_TO_3857.transform(
        df_wind_turbines["lat"], df_wind_turbines["lon"]
    )
    df_wind_turbines["coordinates"] = tuple(zip(df_wind_turbines["lat"], df_wind_turbines["lon"]))

    source_wind_turbines = ColumnDataSource(df_wind_turbines)
    
    render_wind_turbines = p.scatter(x="x", y="y", source=source_wind_turbines, size=8, **marker_options)

    # HoverTool for wind turbines
    hover_wind_turbines = HoverTool(renderers=[render_wind_turbines])
    hover_wind_turbines.tooltips = [("nodeId", "@id"),
                            ("capacity MW", "@generator_output_electricity"),
                            ("hub_height", "@height_hub"),
                            ("rotor_diameter", "@rotor_diameter"),
                            ("manufacturer", "@manufacturer"),
                            ("model", "@model"),
                            ("(Lat,Lon)", "@coordinates")
                            ]
    p.add_tools(hover_wind_turbines)


    # Plot the substations
    marker_options = {
        "marker": "square",
        "line_width": 1,
        "alpha": 0.8,
        "fill_color": "black",
        "line_color": "red",
    }

    if df_substations is not None:
        df_substations["x"], df_substations["y"] = TRANSFORM_4326_TO_3857.transform(
            df_substations["lat"], df_substations["lon"]
        )
        df_substations["coordinates"] = tuple(zip(df_substations["lat"], df_substations["lon"]))

        source_substations = ColumnDataSource(df_substations)
        
        render_substations = p.scatter(x="x", y="y", source=source_substations, size=12, **marker_options)

        # HoverTool for substations
        hover_substations = HoverTool(renderers=[render_substations])
        hover_substations.tooltips = [("nodeId", "@id"),
                                ("name", "@name"),
                                ("voltage", "@voltage"),
                                ("(Lat,Lon)", "@coordinates")
                                ]
        p.add_tools(hover_substations)
                


    url = "https://www.openstreetmap.org/node/@id/"
    
    render_wind_turbines.selection_glyph = None
    render_wind_turbines.nonselection_glyph = None
    taptool_wind_turbines = TapTool(renderers=[render_wind_turbines])
    taptool_wind_turbines.callback = OpenURL(url=url)
    p.add_tools(taptool_wind_turbines)


    return p

In [16]:
import re

df_power_line = get_power_line_data(area,"power_lines")
df_wind_turbines = get_wind_turbine_data(area,"wind_turbines")
df_substations = get_substation_data(area,"substations")

df_power_line = df_power_line.rename(columns=lambda x: re.sub(':','_',x))
df_wind_turbines = df_wind_turbines.rename(columns=lambda x: re.sub(':','_',x))
df_substations = df_substations.rename(columns=lambda x: re.sub(':','_',x))

In [17]:
df_plants = get_plant_data(area,"plants")

In [18]:
df_plants

,type,id,bounds,nodes,geometry,tags,alt_name,name,operator,operator:wikidata,...,note,seamark:restricted_area:category,seamark:restricted_area:restriction,website,plant:method,seamark:source,lat,lon,key,tag
0,way,114331528,"{'minlat': 51.6302768, 'minlon': 2.771835, 'ma...","[1295371750, 5807302750, 5807302751, 129537174...","[{'lat': 51.6861789, 'lon': 2.8382826}, {'lat'...","{'alt_name': 'Belwind phase 1', 'name': 'Belwi...",Belwind phase 1,Belwind,Parkwind,Q66124532,...,NaN,NaN,NaN,NaN,NaN,NaN,51.674885,2.816660,plant:source,wind
1,way,259551141,"{'minlat': 51.592, 'minlon': 2.849333, 'maxlat...","[6462934429, 6462934430, 2649782806, 264978277...","[{'lat': 51.611667, 'lon': 2.9305}, {'lat': 51...","{'access': 'no', 'area': 'yes', 'name': 'North...",NaN,Northwind,Parkwind,Q66124532,...,NaN,NaN,NaN,NaN,NaN,NaN,51.609118,2.899944,plant:source,wind
2,way,431366037,"{'minlat': 51.57027, 'minlon': 3.023953, 'maxl...","[4305707824, 4305707823, 4305707822, 430570782...","[{'lat': 51.69816, 'lon': 3.048502}, {'lat': 5...","{'access': 'no', 'name': 'Borssele II', 'note'...",NaN,Borssele II,Ørsted,NaN,...,"not so sure for the turbine names, should be B...",safety_zone,no_entry,https://orsted.nl/onze-windparken/borssele-1-a...,NaN,NaN,51.648368,3.053543,plant:source,wind
3,way,632999580,"{'minlat': 51.5581098, 'minlon': 2.877, 'maxla...","[2649782772, 2649782774, 2649782781, 264978279...","[{'lat': 51.5913333, 'lon': 2.877}, {'lat': 51...","{'access': 'no', 'area': 'yes', 'name': 'Rente...",NaN,Rentel,NaN,NaN,...,NaN,NaN,NaN,https://rentel.be/,wind_turbine,“MSC – Coastal Division – Flemish Hydrography”...,51.584627,2.937512,plant:source,wind
4,way,685961778,"{'minlat': 51.6875534, 'minlon': 2.6975745, 'm...","[8486595145, 8486595144, 6429140680, 640946955...","[{'lat': 51.7051882, 'lon': 2.6975745}, {'lat'...","{'access': 'no', 'area': 'yes', 'name': 'Merma...",NaN,Mermaid,Otary,NaN,...,NaN,NaN,NaN,NaN,wind_turbine,“MSC – Coastal Division – Flemish Hydrography”...,51.717020,2.737268,plant:source,wind
5,way,688991076,"{'minlat': 51.599667, 'minlon': 2.810167, 'max...","[6462934434, 6462934433, 6462934431, 646293443...","[{'lat': 51.667667, 'lon': 2.878667}, {'lat': ...","{'access': 'no', 'area': 'yes', 'name': 'Seast...",NaN,Seastar,Otary,NaN,...,NaN,NaN,NaN,NaN,wind_turbine,“MSC – Coastal Division – Flemish Hydrography”...,51.639337,2.864295,plant:source,wind
6,way,789978857,"{'minlat': 51.484, 'minlon': 2.929, 'maxlat': ...","[7385889448, 7385889447, 7385889446, 738588944...","[{'lat': 51.5115, 'lon': 2.929}, {'lat': 51.51...","{'access': 'no', 'area': 'yes', 'name': 'North...",NaN,Norther,Elicio;Eneco;DGE,NaN,...,NaN,NaN,NaN,https://www.norther.be/,wind_turbine,“MSC – Coastal Division – Flemish Hydrography”...,51.539013,3.007114,plant:source,wind
7,way,801038528,"{'minlat': 51.6481904, 'minlon': 2.7138678, 'm...","[7492546909, 7492546908, 7492546907, 749254690...","[{'lat': 51.7148333, 'lon': 2.7945}, {'lat': 5...","{'access': 'no', 'area': 'yes', 'name': 'North...",NaN,Northwester 2,Parkwind,Q66124532,...,not all 23 turbines visible on Sentinel-1 SAR,NaN,NaN,NaN,NaN,“MSC – Coastal Division – Flemish Hydrography”...,51.687659,2.760177,plant:source,wind
8,way,1033837375,"{'minlat': 51.7050581, 'minlon': 2.9932686, 'm...","[9524444420, 9524444419, 9524444418, 952444441...","[{'lat': 51.708665, 'lon': 3.0133438}, {'lat':...","{'access': 'no', 'name': 'Borssele V', 'plant:...",NaN,Borssele V,NaN,NaN,...,NaN,safety_zone,no_entry,NaN,NaN,NaN,51.709058,3.005314,plant:source,wind


In [19]:
output_file("wind_farms.html")

In [20]:
set(df_power_line["voltage"])

{'150000', '220000', '33000', '450000', '66000', nan}

In [21]:
color_mapping = dict(zip(['33000', '150000', '220000', '450000'],["green","orange","yellow","red"]))

df_power_line["color"] = df_power_line["voltage"].map(color_mapping)
df_power_line["color"] = df_power_line["color"].fillna("black")


In [22]:
wind_farms = plot_wind_farms(df_wind_turbines,df_power_line,df_substations)

wind_farms.title.text = "Offshore wind farms in OpenStreetMap"

In [23]:
x_coords_view,y_coords_view = TRANSFORM_4326_TO_3857.transform([51.45,51.85],[2.65,3.2])

wind_farms.x_range = Range1d(min(x_coords_view),max(x_coords_view))
wind_farms.y_range = Range1d(min(y_coords_view),max(y_coords_view))

wind_farms.axis.visible=False
wind_farms.grid.visible = False

In [24]:
show(wind_farms)

## UK rivers coloured by name

In [25]:
# uk_colors.add_tools(HoverTool(tooltips=[("River","@name")]))